# Zip Code Analysis

Analyzing every zip code in the United States by looking at Census Block Groups that are entirely contained or more tha 50% contained by a Zip Code Tabluation Area.

This projects uses CARTOframes and the CARTO Data Observatory to import boundaries and US Census Data.

In [2]:
import pandas as pd
import seaborn as sns
import cartoframes

from cartoframes import Credentials

from cartoframes.data import Dataset
from cartoframes.auth import set_default_context, Context
from cartoframes.viz import Map, Layer
from cartoframes.viz.helpers import color_continuous_layer, color_category_layer

username = 'mbforrcdb' # <-- insert your username here
api_key = 'b93d8a3b37e53fc20361c5ce5369b2c7b594bd44'# <-- insert your API key here

cc = Context('https://{}.carto.com/'.format(username), api_key)
set_default_context(cc)

# Import Census Block Groups

In [ ]:
cc.execute('''
WITH meta AS (SELECT OBS_GetMeta(
  ST_makeenvelope(-179.5,-14.8,-64.4,71.6, 4326),
  '[{"geom_id": "us.census.tiger.block_group_clipped"}]'
) meta)
INSERT INTO income_bgs (the_geom, name)
SELECT (data->0->>'value')::Geometry the_geom, 
       (data->0->>'geomref') geomref
FROM OBS_GetData(Array[(ST_makeenvelope(-179.5,-14.8,-64.4,71.6, 4326), 1)::geomval],
  (SELECT meta FROM meta), false)

''')

In [ ]:
cc.execute('''
ALTER TABLE income_bgs 
ADD COLUMN median_income numeric,
ADD COLUMN total_population numeric,
ADD COLUMN population_density numeric
''')

# Add data for Median Income in the Last 12 Months, Total Population, and Population Density

In [ ]:
cc.execute('''
WITH meta AS (SELECT OBS_GetMeta(
  ST_SetSRID(ST_Extent(the_geom), 4326),
  '[{"numer_id": "us.census.acs.B19013001",
     "normalization": "prenormalized",
     "denom_id": "",
     "numer_timespan": "2011 - 2015",
     "geom_id": "us.census.tiger.block_group_clipped"},
     {"numer_id": "us.census.acs.B01003001",
     "normalization": "prenormalized",
     "denom_id": "",
     "numer_timespan": "2011 - 2015",
     "geom_id": "us.census.tiger.block_group_clipped"},
     {"numer_id": "us.census.acs.B01003001",
     "normalization": "areaNormalized",
     "denom_id": "",
     "numer_timespan": "2011 - 2015",
     "geom_id": "us.census.tiger.block_group_clipped"}]'
) meta from income_bgs),
data as (
SELECT id as name, 
(data->0->>'value')::Numeric a,
(data->1->>'value')::Numeric b,
(data->2->>'value')::Numeric c
FROM OBS_GetData((SELECT ARRAY_AGG(name) from income_bgs),
  (SELECT meta FROM meta)))
UPDATE income_bgs
SET (median_income, total_population, population_density) =
    (SELECT a, b, c FROM data WHERE data.name = income_bgs.name)
''')

# Add in ZCTA Boundaries

In [ ]:
cc.execute('''
WITH meta AS (SELECT OBS_GetMeta(
  ST_makeenvelope(-179.5,-14.8,-64.4,71.6, 4326),
  '[{"geom_id": "us.census.tiger.zcta5_clipped"}]'
) meta)
INSERT INTO income_zips (the_geom, name)
SELECT (data->0->>'value')::Geometry the_geom, 
       (data->0->>'geomref') geomref
FROM OBS_GetData(Array[(ST_makeenvelope(-179.5,-14.8,-64.4,71.6, 4326), 1)::geomval],
  (SELECT meta FROM meta), false)

''')

# Explore Block Groups Intersecting One Zip Code

In [3]:
nyc = Dataset.from_query('''
SELECT 
b.* 
FROM
income_bgs b, income_zips z
WHERE ST_Intersects(z.the_geom, b.the_geom)
AND z.name = '10001'
''')

In [4]:
nyc = nyc.download()

In [5]:
nyc.describe()

,median_income,total_population,population_density
count,28.000000,31.000000,31.000000
mean,106491.250000,1669.548387,30777.052393
std,38959.988292,1075.017018,33053.146775
min,19697.000000,186.000000,1057.193323
25%,87342.500000,1014.500000,15591.597434
50%,106442.000000,1433.000000,20667.715381
75%,133781.250000,1892.500000,34341.605926
max,159821.000000,4938.000000,178472.023424


# Build a query to replicate a describe method

In [6]:
nyc = Dataset.from_query('''
SELECT 
    z.*,
    min(b.median_income),
    max(b.median_income),
    percentile_disc(0.25) within group (order by b.median_income) as p_25,
    percentile_disc(0.5) within group (order by b.median_income) as mean,
    percentile_disc(0.75) within group (order by b.median_income) as p_75,
    stddev_pop(b.median_income)
FROM
    income_bgs b, income_zips z
WHERE ST_Intersects(z.the_geom, b.the_geom)
AND (st_area(st_intersection(z.the_geom, b.the_geom))/st_area(b.the_geom)) > .5
AND z.name = '10001'
GROUP BY z.cartodb_id
''')

In [7]:
nyc = nyc.download()
nyc

,the_geom,name,description,min,max,p_25,mean,p_75,stddev_pop
cartodb_id,,,,,,,,,
2568,0103000020E6100000010000002C0000008805BEA25B80...,10001,NaN,19697.0,159821.0,81146.0,104097.0,133125.0,40738.508151


# Create a map for this Zip Code and Intersecting Block Groups

In [8]:
zip_code = 10001

bgs = Dataset.from_query(f'''
SELECT 
    b.*
FROM
    income_bgs b, income_zips z
WHERE ST_Intersects(z.the_geom, b.the_geom)
AND (st_area(st_intersection(z.the_geom, b.the_geom))/st_area(b.the_geom)) > .5
AND z.name = '{zip_code}'
''')

zip = Dataset.from_query(f'''
SELECT 
    *
FROM
    income_zips 
WHERE name = '{zip_code}'
''')

In [9]:
Map(layers=[
    color_continuous_layer(bgs, 'median_income', title='Median Income', palette='Sunset'),
    Layer(zip,
         '''color: opacity(rgb(145, 230, 167), 0);
        strokeColor: rgb(145, 230, 167)
        strokeWidth: 6
         ''')
    ])

In [10]:
nyc

,the_geom,name,description,min,max,p_25,mean,p_75,stddev_pop
cartodb_id,,,,,,,,,
2568,0103000020E6100000010000002C0000008805BEA25B80...,10001,NaN,19697.0,159821.0,81146.0,104097.0,133125.0,40738.508151


# Create a table to evaluate inequality in Zip Codes

In [ ]:
nyc_final = cc.execute('''
CREATE TABLE zips_income_step_2_1 AS
SELECT 
    z.*,
    min(b.median_income),
    max(b.median_income),
    sum(b.total_population) as total_pop,
    percentile_disc(0.25) within group (order by b.median_income) as p_25,
    percentilae_disc(0.5) within group (order by b.median_income) as mean,
    percentile_disc(0.75) within group (order by b.median_income) as p_75,
    stddev_pop(b.median_income)
FROM
    income_zips z
LEFT JOIN income_bgs b 
ON ST_Intersects(z.the_geom, b.the_geom)
WHERE b.total_population > 0
AND (st_area(st_intersection(z.the_geom, b.the_geom))/st_area(b.the_geom)) > .5
GROUP BY z.cartodb_id
''')

In [11]:
zip_code_analysis = Dataset.from_query('''
SELECT *,
coalesce(min - max, 0) AS dif
FROM zips_income_step_2_1
''')

In [12]:
z = zip_code_analysis.download(decode_geom=True)
z.head(2)

,geometry,name,description,min,max,total_pop,p_25,mean,p_75,stddev_pop,dif
cartodb_id,,,,,,,,,,,
1,"POLYGON ((-66.68155199999998 18.138412, -66.68...",00601,NaN,5881.0,15000.0,18962.0,8504.0,10000.0,14833.0,2892.955531,-9119.0
2,POLYGON ((-67.22742299999999 18.34871099999999...,00602,NaN,7534.0,37578.0,40603.0,13099.0,14946.0,18750.0,6821.665073,-30044.0


# Prepare data for Moran's I

In [13]:
z_for_moran = Dataset.from_query('''
SELECT 
ST_Centroid(the_geom) as the_geom,
coalesce(max - min, 0) AS dif,
name
FROM zips_income_step_2_1
WHERE the_geom &&
ST_MakeEnvelope(-128.6,23.0,-64.4,51.3, 4326)
''')

In [14]:
z = z_for_moran.download(decode_geom=True)

In [15]:
import geopandas as gpd
z = gpd.GeoDataFrame(z)

In [16]:
z.head()

,geometry,dif,name
0,POINT (-72.62735604216334 42.06246515244545),76933.0,01001
1,POINT (-72.46451219153876 42.37473097374864),147467.0,01002
2,POINT (-72.5247589905164 42.3919061673208),0.0,01003
3,POINT (-72.10611700927198 42.42009505028397),53355.0,01005
4,POINT (-72.40220444660021 42.27718810348262),59782.0,01007


# Visualize Results

In [17]:
Map(color_continuous_layer(zip_code_analysis, 'dif', title='Difference Between Min/Max Median Income in ZCTA', palette='Sunset'))

# Create spatial weights

First we need to evaluate the spatial relationships between all the different buildings. Since these geometries do not touch, we want to use the KNN weights from PySAL:

https://libpysal.readthedocs.io/en/latest/generated/libpysal.weights.Queen.html#libpysal.weights.Queen

In [18]:
import libpysal

W = libpysal.weights.KNN.from_dataframe(z, k=5)
W.transform = 'r'

# Moran's I Local

To identify the significant clusters, we will use the Moran's I Local analysis from PySAL to identify clusters of high sale prices. Spatial autocorrelation as described by the PySAL examples:

*The concept of spatial autocorrelation relates to the combination of two types of similarity: spatial similarity and attribute similarity. Although there are many different measures of spatial autocorrelation, they all combine these two types of simmilarity into a summary measure.*

http://darribas.org/gds_scipy16/ipynb_md/04_esda.html
https://nbviewer.jupyter.org/github/pysal/esda/blob/master/notebooks/Spatial%20Autocorrelation%20for%20Areal%20Unit%20Data.ipynb

In [19]:
lag = libpysal.weights.lag_spatial(W, z.dif)

In [20]:
import esda
moran = esda.Moran_Local(z.dif, W, transformation = "r")

In [21]:
moran.q[10:100]

array([2, 3, 4, 1, 3, 1, 3, 3, 2, 3, 2, 2, 3, 3, 1, 3, 3, 3, 1, 4, 4, 4,
       2, 4, 3, 3, 3, 1, 1, 1, 2, 3, 4, 3, 1, 1, 3, 4, 3, 3, 2, 1, 2, 1,
       1, 1, 1, 1, 1, 2, 1, 2, 4, 3, 3, 3, 3, 3, 4, 3, 3, 2, 4, 1, 3, 4,
       3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 3, 3, 2, 3, 3,
       3, 3])

# Similarity

From PySAL Docs:

**p_sim : array**

(if permutations>0) p-values based on permutations (one-sided) null: spatial randomness alternative: the observed Ii is further away or extreme from the median of simulated values. It is either extremelyi high or extremely low in the distribution of simulated Is.

In [22]:
moran.p_sim

array([0.31 , 0.131, 0.379, ..., 0.016, 0.406, 0.014])

From PySAL Docs:

**p_z_sim : array**

(if permutations>0) p-values based on standard normal approximation from permutations (one-sided) for two-sided tests, these values should be multiplied by 2

In [23]:
moran.p_z_sim

array([0.32640673, 0.14235094, 0.40448374, ..., 0.00848765, 0.37148787,
       0.00302667])

In [24]:
data = z.dif

In [25]:
sig = 1 * (moran.p_sim < 0.05)
HH = 1 * (sig * moran.q==1)
LL = 3 * (sig * moran.q==3)
LH = 2 * (sig * moran.q==2)
HL = 4 * (sig * moran.q==4)
spots = HH + LL + LH + HL
spots

array([0, 0, 0, ..., 1, 0, 1])

In [26]:
spot_labels = [ '0 Non-Significant', 'HH - Hot Spot', 'LH - Donut', 'LL - Cold Spot', 'HL - Diamond']
labels = [spot_labels[i] for i in spots]

In [27]:
moran_to_carto = z.assign(cl=labels, p_sim = moran.p_sim, p_z_sim = moran.p_z_sim)
moran_to_carto.head(2)

,geometry,dif,name,cl,p_sim,p_z_sim
0,POINT (-72.62735604216334 42.06246515244545),76933.0,01001,0 Non-Significant,0.310,0.326407
1,POINT (-72.46451219153876 42.37473097374864),147467.0,01002,0 Non-Significant,0.131,0.142351


In [ ]:
to_carto = Dataset.from_dataframe(moran_to_carto)

In [ ]:
to_carto.upload(table_name='zips_morans_i', if_exists='replace')

In [ ]:
cc.execute('''
UPDATE zips_morans_i 
SET the_geom = z.the_geom
FROM zips_income_step_2_1 z
WHERE zips_morans_i.name = z.name
''')

In [28]:
to_map = Dataset.from_table('zips_morans_i')

In [29]:
Map(color_category_layer(to_map, 'cl'))